In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, utils
from torch.utils import data
import matplotlib.pyplot as plt
import numpy as np

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

EPOCHS = 30
BATCH_SIZE = 64

transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = datasets.FashionMNIST(
    root = "./data/",
    train = True,
    download = True,
    transform = transform
)

testset = datasets.FashionMNIST(
    root = "./data/",
    train = False,
    download = True,
    transform = transform
)

train_loader = data.DataLoader(  # 에폭마다 네트워크에 데이터를 주입해줌
    dataset = trainset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = data.DataLoader(
    dataset = testset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net().to(DEVICE)  # cpu 혹은 cuda로 네트워크를 보냄
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [14]:
def train(model, train_loader, optimizer):
    model.train()  # 모델을 학습 모드로 전환, 드롭아웃 등의 레이어에서 동작이 다름

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)  # 데이터도 처리할 디바이스로 보내야 함
        optimizer.zero_grad()  # 매 반복마다 기울기를 새로 계산해야 함
        output = model(data)
        loss = F.cross_entropy(output, target)  # 결과물과 정답 사이 크로스엔트로피 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 수정


def evaluate(model, test_loader):
    model.eval()  # 모델을 평가 모드로 전환
    test_loss = 0
    correct = 0

    with torch.no_grad():  # 평가시엔 기울기 계산 필요없음
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target, reduction="sum").item()

            pred = output.max(1, keepdim=True)[1]  # 모델의 예측 클래스
            correct += pred.eq(target.view_as(pred)).sum().item()  # 정답이면 +1

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    return test_loss, test_accuracy


In [15]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss: 0.8551, Accuracy: 67.41%
[2] Test Loss: 0.6732, Accuracy: 75.62%
[3] Test Loss: 0.5994, Accuracy: 78.62%
[4] Test Loss: 0.5492, Accuracy: 80.55%
[5] Test Loss: 0.5153, Accuracy: 81.66%
[6] Test Loss: 0.5035, Accuracy: 82.26%
[7] Test Loss: 0.4830, Accuracy: 83.11%
[8] Test Loss: 0.4666, Accuracy: 83.57%
[9] Test Loss: 0.4741, Accuracy: 83.18%
[10] Test Loss: 0.4554, Accuracy: 83.91%
[11] Test Loss: 0.4555, Accuracy: 83.84%
[12] Test Loss: 0.4428, Accuracy: 84.27%
[13] Test Loss: 0.4338, Accuracy: 84.68%
[14] Test Loss: 0.4344, Accuracy: 84.58%
[15] Test Loss: 0.4205, Accuracy: 85.08%
[16] Test Loss: 0.4221, Accuracy: 85.13%
[17] Test Loss: 0.4226, Accuracy: 84.95%
[18] Test Loss: 0.4081, Accuracy: 85.62%
[19] Test Loss: 0.4150, Accuracy: 85.17%
[20] Test Loss: 0.4118, Accuracy: 85.25%
[21] Test Loss: 0.4072, Accuracy: 85.50%
[22] Test Loss: 0.3984, Accuracy: 85.50%
[23] Test Loss: 0.3901, Accuracy: 86.07%
[24] Test Loss: 0.3987, Accuracy: 85.83%
[25] Test Loss: 0.3883, A